# Compactação de Dados com Apache Iceberg

Este notebook demonstra como realizar compactação de dados no Apache Iceberg, uma operação essencial para manter performance e otimizar o armazenamento em data lakes.

## Contexto e Objetivos:

### Problema dos Arquivos Pequenos:
- **Fragmentação**: Múltiplas inserções criam muitos arquivos pequenos
- **Performance**: Overhead de metadados e I/O ineficiente
- **Custos**: Maior número de operações de leitura
- **Escalabilidade**: Degradação com crescimento dos dados

### Benefícios da Compactação:
- **Performance**: Menos arquivos = menos overhead de I/O
- **Eficiência**: Melhor compressão e utilização de recursos
- **Manutenção**: Redução de metadados e complexidade
- **Custos**: Otimização de armazenamento e processamento

### Estratégias de Compactação:
1. **Rewrite Data Files**: Reorganização completa dos arquivos
2. **Bin Packing**: Agrupamento eficiente de arquivos pequenos
3. **Z-Order**: Otimização para queries com múltiplas dimensões
4. **Expire Snapshots**: Limpeza de versões antigas

## Setup do Ambiente

In [1]:
# Para o Spark se estiver rodando
try:
    spark.stop()
except:
    pass

# Configuração do Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark-3.3.0-bin-hadoop3"

import findspark
findspark.init('/opt/spark-3.3.0-bin-hadoop3')

from pyspark.sql import SparkSession

# Sessão Spark
spark = SparkSession.builder \
    .appName("IcebergCompaction") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.hadoop_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.hadoop_catalog.type", "hadoop") \
    .config("spark.sql.catalog.hadoop_catalog.warehouse", "file:///home/tavares/warehouse") \
    .config("spark.sql.default.catalog", "hadoop_catalog") \
    .config("spark.hadoop.fs.defaultFS", "file:///") \
    .getOrCreate()

print(f"Warehouse configurado para: {spark.conf.get('spark.sql.catalog.hadoop_catalog.warehouse')}")

Warehouse configurado para: file:///home/tavares/warehouse


## 1. Criação da Tabela de Exemplo

Vamos criar uma tabela que será usada para demonstrar os conceitos de compactação.

In [2]:
# Exclui se existir
spark.sql("DROP TABLE IF EXISTS hadoop_catalog.default.vendas")

# Cria a tabela Vendas
spark.sql("""
    CREATE TABLE hadoop_catalog.default.vendas (
        id INT,
        produto STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
""")

DataFrame[]

## 2. Simulação do Problema - Criando Arquivos Pequenos

Vamos simular um cenário comum onde múltiplas inserções pequenas criam fragmentação de arquivos. Cada inserção individual criará um arquivo separado, demonstrando o problema que a compactação resolve.

In [3]:
# Inserir dados em lotes pequenos para criar vários arquivos
# Simulando inserções frequentes de sistemas transacionais

# 10 Lotes de Dados (cada um criará um arquivo separado)
for i in range(1, 11):
    spark.sql(f"""
        INSERT INTO hadoop_catalog.default.vendas VALUES
        ({i}, 'Produto {i}', {i * 2}, {i * 10.0}, DATE('2024-11-{i:02d}'))
    """)

print("10 inserções individuais realizadas - cada uma criou um arquivo separado")

10 inserções individuais realizadas - cada uma criou um arquivo separado


## 3. Análise do Estado Antes da Compactação

Vamos analisar quantos arquivos foram criados e como os dados estão distribuídos.

In [4]:
# Analisar arquivos antes da compactação
print("=== ANTES DA COMPACTAÇÃO ===")

# Contar arquivos únicos
df_files_before = spark.sql("""
    SELECT
        COUNT(*) AS registros,
        input_file_name() AS arquivo
    FROM hadoop_catalog.default.vendas
    GROUP BY input_file_name()
""")

print("Distribuição de registros por arquivo:")
df_files_before.show(truncate=False)

num_arquivos_antes = df_files_before.count()
total_registros = spark.sql("SELECT COUNT(*) as total FROM hadoop_catalog.default.vendas").collect()[0][0]

print(f"\nResumo:")
print(f"- Número total de arquivos: {num_arquivos_antes}")
print(f"- Total de registros: {total_registros}")
print(f"- Média de registros por arquivo: {total_registros / num_arquivos_antes:.1f}")

=== ANTES DA COMPACTAÇÃO ===
Distribuição de registros por arquivo:
+---------+-------------------------------------------------------------------------------------------------------------+
|registros|arquivo                                                                                                      |
+---------+-------------------------------------------------------------------------------------------------------------+
|1        |file:/home/tavares/warehouse/default/vendas/data/00000-3-baf6144c-28f4-42d3-a539-b15ca2ef849e-0-00001.parquet|
|1        |file:/home/tavares/warehouse/default/vendas/data/00000-9-ed871109-12af-460e-a995-0cb706b055aa-0-00001.parquet|
|1        |file:/home/tavares/warehouse/default/vendas/data/00000-7-61eacd63-16fd-423c-a00d-30b039dc3c86-0-00001.parquet|
|1        |file:/home/tavares/warehouse/default/vendas/data/00000-1-47ecbacb-9403-4475-85db-92b80dcedd3c-0-00001.parquet|
|1        |file:/home/tavares/warehouse/default/vendas/data/00000-0-6acfa4a5-c

## 4. Verificação de Informações dos Arquivos

Vamos usar as tabelas de metadados do Iceberg para obter informações detalhadas sobre os arquivos.

In [5]:
# Informações detalhadas dos arquivos usando metadados Iceberg
print("=== INFORMAÇÕES DOS ARQUIVOS (METADADOS ICEBERG) ===")

spark.sql("""
    SELECT 
        file_path,
        file_format,
        record_count,
        file_size_in_bytes,
        ROUND(file_size_in_bytes / 1024.0, 2) as file_size_kb
    FROM hadoop_catalog.default.vendas.files
    ORDER BY file_path
""").show(truncate=False)

# Estatísticas agregadas
spark.sql("""
    SELECT 
        COUNT(*) as total_files,
        SUM(record_count) as total_records,
        AVG(record_count) as avg_records_per_file,
        SUM(file_size_in_bytes) as total_size_bytes,
        AVG(file_size_in_bytes) as avg_file_size_bytes
    FROM hadoop_catalog.default.vendas.files
""").show()

=== INFORMAÇÕES DOS ARQUIVOS (METADADOS ICEBERG) ===
+-------------------------------------------------------------------------------------------------------------+-----------+------------+------------------+------------+
|file_path                                                                                                    |file_format|record_count|file_size_in_bytes|file_size_kb|
+-------------------------------------------------------------------------------------------------------------+-----------+------------+------------------+------------+
|file:/home/tavares/warehouse/default/vendas/data/00000-0-6acfa4a5-c43d-485f-9001-2e431b0da094-0-00001.parquet|PARQUET    |1           |1393              |1.36        |
|file:/home/tavares/warehouse/default/vendas/data/00000-1-47ecbacb-9403-4475-85db-92b80dcedd3c-0-00001.parquet|PARQUET    |1           |1393              |1.36        |
|file:/home/tavares/warehouse/default/vendas/data/00000-2-18cb0dc8-7683-4a5d-bead-e967d9e32da2-0-00001

## 5. Executando a Compactação

Agora vamos executar a compactação usando o procedimento `rewrite_data_files` do Iceberg. Esta operação reorganizará os arquivos pequenos em arquivos maiores e mais eficientes.

In [6]:
# Configurar tamanho máximo de registros por arquivo
spark.conf.set("spark.sql.files.maxRecordsPerFile", 1000)

print("Executando compactação...")

# Compactação com procedimento 'rewrite_data_files'
result = spark.sql("""
    CALL hadoop_catalog.system.rewrite_data_files(
        table => 'default.vendas'
    )
""")

# Mostrar resultado da compactação
print("\nResultado da compactação:")
result.show()

print("Compactação concluída!")

Executando compactação...

Resultado da compactação:
+--------------------------+----------------------+---------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|
+--------------------------+----------------------+---------------------+
|                        10|                     1|                13934|
+--------------------------+----------------------+---------------------+

Compactação concluída!


## 6. Análise do Estado Após a Compactação

Vamos verificar como a compactação afetou a estrutura dos arquivos.

In [7]:
# Analisar arquivos após a compactação
print("=== APÓS A COMPACTAÇÃO ===")

df_files_after = spark.sql("""
    SELECT
        COUNT(*) AS registros,
        input_file_name() AS arquivo
    FROM hadoop_catalog.default.vendas
    GROUP BY input_file_name()
""")

print("Nova distribuição de registros por arquivo:")
df_files_after.show(truncate=False)

num_arquivos_depois = df_files_after.count()
total_registros_depois = spark.sql("SELECT COUNT(*) as total FROM hadoop_catalog.default.vendas").collect()[0][0]

print(f"\nResumo após compactação:")
print(f"- Número total de arquivos: {num_arquivos_depois}")
print(f"- Total de registros: {total_registros_depois}")
print(f"- Média de registros por arquivo: {total_registros_depois / num_arquivos_depois:.1f}")

print(f"\nMelhoria:")
print(f"- Redução de arquivos: {num_arquivos_antes} → {num_arquivos_depois} ({((num_arquivos_antes - num_arquivos_depois) / num_arquivos_antes * 100):.1f}% redução)")
print(f"- Dados preservados: {total_registros == total_registros_depois}")

=== APÓS A COMPACTAÇÃO ===
Nova distribuição de registros por arquivo:
+---------+--------------------------------------------------------------------------------------------------------------+
|registros|arquivo                                                                                                       |
+---------+--------------------------------------------------------------------------------------------------------------+
|10       |file:/home/tavares/warehouse/default/vendas/data/00000-19-24e176cf-8bbd-40b1-b4df-35fc392418d6-0-00001.parquet|
+---------+--------------------------------------------------------------------------------------------------------------+


Resumo após compactação:
- Número total de arquivos: 1
- Total de registros: 10
- Média de registros por arquivo: 10.0

Melhoria:
- Redução de arquivos: 10 → 1 (90.0% redução)
- Dados preservados: True


## 7. Verificação dos Snapshots

A compactação cria um novo snapshot. Vamos verificar o histórico de snapshots.

In [8]:
# Verificar snapshots criados
print("=== HISTÓRICO DE SNAPSHOTS ===")

spark.sql("""
    SELECT 
        snapshot_id,
        committed_at,
        operation,
        summary
    FROM hadoop_catalog.default.vendas.snapshots 
    ORDER BY committed_at
""").show(truncate=False)

print("\nObserve que a compactação criou um novo snapshot com operação 'replace'")

=== HISTÓRICO DE SNAPSHOTS ===
+-------------------+-----------------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|snapshot_id        |committed_at           |operation|summary                                                                                                                                                                                                                                                                                                                                                              

## 8. Limpeza de Snapshots Antigos

Após a compactação, podemos limpar snapshots antigos para liberar espaço em disco. Esta operação remove arquivos que não são mais referenciados.

In [9]:
# Limpeza de snapshots antigos
print("Executando limpeza de snapshots antigos...")

# Manter apenas o último snapshot
cleanup_result = spark.sql("""
    CALL hadoop_catalog.system.expire_snapshots(
        table => 'default.vendas',
        retain_last => 1
    )
""")

print("\nResultado da limpeza:")
cleanup_result.show()

print("Limpeza concluída! Arquivos antigos foram removidos.")

Executando limpeza de snapshots antigos...

Resultado da limpeza:
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+
|deleted_data_files_count|deleted_position_delete_files_count|deleted_equality_delete_files_count|deleted_manifest_files_count|deleted_manifest_lists_count|deleted_statistics_files_count|
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+------------------------------+
|                       0|                                  0|                                  0|                           0|                           0|                             0|
+------------------------+-----------------------------------+-----------------------------------+----------------------------+----------------------------+--------------------------

## 9. Verificação Final

Vamos fazer uma verificação final para confirmar que os dados estão íntegros e a estrutura foi otimizada.

In [10]:
# Verificação final dos dados
print("=== VERIFICAÇÃO FINAL ===")

# Dados ainda estão corretos?
print("Dados na tabela:")
spark.sql("SELECT * FROM hadoop_catalog.default.vendas ORDER BY id").show()

# Informações finais dos arquivos
print("\nInformações finais dos arquivos:")
spark.sql("""
    SELECT 
        COUNT(*) as total_files,
        SUM(record_count) as total_records,
        AVG(record_count) as avg_records_per_file,
        ROUND(SUM(file_size_in_bytes) / 1024.0, 2) as total_size_kb,
        ROUND(AVG(file_size_in_bytes) / 1024.0, 2) as avg_file_size_kb
    FROM hadoop_catalog.default.vendas.files
""").show()

# Snapshots restantes
print("\nSnapshots restantes após limpeza:")
spark.sql("""
    SELECT 
        snapshot_id,
        committed_at,
        operation
    FROM hadoop_catalog.default.vendas.snapshots 
    ORDER BY committed_at
""").show(truncate=False)

=== VERIFICAÇÃO FINAL ===
Dados na tabela:
+---+----------+----------+-----+----------+
| id|   produto|quantidade|preco|data_venda|
+---+----------+----------+-----+----------+
|  1| Produto 1|         2| 10.0|2024-11-01|
|  2| Produto 2|         4| 20.0|2024-11-02|
|  3| Produto 3|         6| 30.0|2024-11-03|
|  4| Produto 4|         8| 40.0|2024-11-04|
|  5| Produto 5|        10| 50.0|2024-11-05|
|  6| Produto 6|        12| 60.0|2024-11-06|
|  7| Produto 7|        14| 70.0|2024-11-07|
|  8| Produto 8|        16| 80.0|2024-11-08|
|  9| Produto 9|        18| 90.0|2024-11-09|
| 10|Produto 10|        20|100.0|2024-11-10|
+---+----------+----------+-----+----------+


Informações finais dos arquivos:
+-----------+-------------+--------------------+-------------+----------------+
|total_files|total_records|avg_records_per_file|total_size_kb|avg_file_size_kb|
+-----------+-------------+--------------------+-------------+----------------+
|          1|           10|                10.0|    

## Resumo dos Benefícios da Compactação

### Resultados Demonstrados:

1. **Redução de Arquivos**: De 10 arquivos pequenos para 1 arquivo otimizado
2. **Preservação de Dados**: Todos os registros mantidos intactos
3. **Melhoria de Performance**: Menos overhead de I/O e metadados
4. **Otimização de Armazenamento**: Melhor compressão e utilização de espaço

### Operações de Manutenção Iceberg:

- ✅ **rewrite_data_files**: Compactação de arquivos pequenos
- ✅ **expire_snapshots**: Limpeza de versões antigas
- 🔄 **rewrite_manifests**: Otimização de manifests
- 🔄 **remove_orphan_files**: Remoção de arquivos órfãos

### Estratégias de Compactação:

1. **Automática**: Configurar triggers baseados em métricas
2. **Periódica**: Jobs schedulados (diário, semanal)
3. **Baseada em Eventos**: Após grandes cargas de dados
4. **Condicional**: Quando número de arquivos excede threshold

### Considerações para Produção:

- **Timing**: Executar durante janelas de baixa utilização
- **Recursos**: Compactação pode ser intensiva em CPU/I/O
- **Monitoramento**: Acompanhar métricas de arquivos e performance
- **Backup**: Manter snapshots críticos antes da limpeza
- **Testes**: Validar integridade após compactação

### Métricas de Monitoramento:

- Número de arquivos por tabela
- Tamanho médio dos arquivos
- Tempo de execução de queries
- Utilização de armazenamento
- Frequência de compactação necessária